In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler


df = pd.read_csv("park.data.csv")

df = df.drop(columns=['subject#', 'age', 'sex', 'test_time', 'total_UPDRS'])

features = df.drop(columns=['motor_UPDRS'])
target = df['motor_UPDRS']


scaler = StandardScaler()
features_normalized = scaler.fit_transform(features)



In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Initialize and train the regression model
regressor = LinearRegression()
regressor.fit(X_train, y_train)

# Make predictions
y_pred = regressor.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print("Coefficients:", regressor.coef_)

Mean Squared Error: 58.9758648222849
Coefficients: [ 1.81436091e+02 -5.23357730e+04 -2.79564491e+04 -8.07814483e+01
  9.39996586e+03  8.53381486e+01 -4.61212769e+00 -2.00866742e+04
 -1.52118805e+02  1.27336950e+02  6.65698268e+03 -2.25764578e+01
 -4.44755130e-01  1.36652161e+00 -3.04053902e+01  2.15883846e+01]


C:\Users\HP 17\AppData\Local\Temp\ipykernel_8432\3387839528.py:7: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_file)


{'Jitter(%)': 0.036344543289537214, 'Jitter(Abs)': 4.088244802562334, 'Jitter:RAP': 0.00858289410048667, 'Jitter:PPQ5': 0.008749764414009561, 'Jitter:DDP': 7.6011113589991055, 'Shimmer': 1.4387935, 'Shimmer(dB)': 4.9437397718429565, 'Shimmer:APQ3': 0.57763004, 'Shimmer:APQ5': 0.57763004, 'Shimmer:APQ11': 0.57763004, 'Shimmer:DDA': 0.57763004, 'NHR': 0.019870464, 'HNR': 0.9805166788677505, 'RPDE': 0.5106680899162364, 'DFA': 0.6956951440038234, 'PPE': 0.0036222364001466945}


In [13]:
# Convert the extracted features to a DataFrame
amir_features_df = pd.DataFrame([amir_features])

new_prediction = regressor.predict(amir_features_df)
print(f"Prediction for the first line of features: {new_prediction[0]}")

Prediction for the first line of features: -150391.4327351434


In [16]:
amir_features_df = pd.DataFrame([amir_features])


new_prediction = regressor.predict([features.iloc[0]])

print(f"Prediction for the new features: {new_prediction[0]}")

Prediction for the new features: 24.119672496302087


c:\Users\HP 17\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [17]:
pip install praat-parselmouth



   ---------------------------------------- 0.0/8.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.9 MB 279.3 kB/s eta 0:00:32
   ---------------------------------------- 0.1/8.9 MB 521.8 kB/s eta 0:00:17
    --------------------------------------- 0.1/8.9 MB 554.9 kB/s eta 0:00:16
   - -------------------------------------- 0.2/8.9 MB 915.1 kB/s eta 0:00:10
   - -------------------------------------- 0.3/8.9 MB 999.9 kB/s eta 0:00:09
   - -------------------------------------- 0.4/8.9 MB 1.1 MB/s eta 0:00:08
   - -------------------------------------- 0.4/8.9 MB 1.1 MB/s eta 0:00:08
   -- ------------------------------------- 0.5/8.9 MB 1.2 MB/s eta 0:00:08
   -- ------------------------------------- 0.6/8.9 MB 1.2 MB/s eta 0:00:07
   -- ------------------------------------- 0.7/8.9 MB 1.3 MB/s eta 0:00:07
   --- ---------------

In [37]:
import parselmouth
from parselmouth.praat import call
import numpy as np

def extract_voice_features(file_path):
    snd = parselmouth.Sound(file_path)
    
    # Pitch bounds: adjust if necessark
    pitch_min = 75
    pitch_max = 500

    point_process = call(snd, "To PointProcess (periodic, cc)", pitch_min, pitch_max)

    print("PointProcess:", point_process)

    # Jitter (requires PointProcess)
    jitter_local = call([snd, point_process], "Jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
    jitter_local_abs = call([snd, point_process], "Get jitter (local, absolute)", 0, 0, 0.0001, 0.02, 1.3)
    jitter_rap = call([snd, point_process], "Get jitter (rap)", 0, 0, 0.0001, 0.02, 1.3)
    jitter_ppq5 = call([snd, point_process], "Get jitter (ppq5)", 0, 0, 0.0001, 0.02, 1.3)
    jitter_ddp = call([snd, point_process], "Get jitter (ddp)", 0, 0, 0.0001, 0.02, 1.3)

    # Shimmer (requires PointProcess)
    shimmer_local = call([snd, point_process], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    shimmer_db = call([snd, point_process], "Get shimmer (local_dB)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    shimmer_apq3 = call([snd, point_process], "Get shimmer (apq3)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    shimmer_apq5 = call([snd, point_process], "Get shimmer (apq5)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    shimmer_apq11 = call([snd, point_process], "Get shimmer (apq11)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    shimmer_dda = call([snd, point_process], "Get shimmer (dda)", 0, 0, 0.0001, 0.02, 1.3, 1.6)

    # Harmonics-to-noise ratio (HNR)
    harmonicity = call(snd, "To Harmonicity (cc)", 0.01, pitch_min, 0.1, 1.0)
    hnr = call(harmonicity, "Get mean", 0, 0)

    # Noise-to-harmonics ratio (inverse of HNR in some contexts)
    nhr = 1 / (10 ** (hnr / 10)) if hnr != 0 else 0

    # Voice report metrics
    voice_report = call(snd, "Voice report", pitch_min, pitch_max, 0.01, 0.3)

    # Try to extract RPDE, DFA, PPE manually from the voice report
    def extract_metric(report, key):
        for line in report.splitlines():
            if key in line:
                try:
                    return float(line.split(":")[1].strip().split()[0])
                except:
                    return None
        return None

    rpde = extract_metric(voice_report, "RPDE")
    dfa = extract_metric(voice_report, "DFA")
    ppe = extract_metric(voice_report, "PPE")

    return {
        "Jitter (%)": jitter_local,
        "Jitter (Abs)": jitter_local_abs,
        "Jitter:RAP": jitter_rap,
        "Jitter:PPQ5": jitter_ppq5,
        "Jitter:DDP": jitter_ddp,
        "Shimmer": shimmer_local,
        "Shimmer(dB)": shimmer_db,
        "Shimmer:APQ3": shimmer_apq3,
        "Shimmer:APQ5": shimmer_apq5,
        "Shimmer:APQ11": shimmer_apq11,
        "Shimmer:DDA": shimmer_dda,
        "NHR": nhr,
        "HNR": hnr,
        "RPDE": rpde,
        "DFA": dfa,
        "PPE": ppe
    }

features = extract_voice_features("amir_clean.wav")
for k, v in features.items():
    print(f"{k}: {v}")


PointProcess: Object type: PointProcess
Object name: untitled
Date: Tue Apr 29 13:40:06 2025

Time domain:
   Start time: 0 seconds
   End time: 1.28 seconds
   Total duration: 1.28 seconds
Number of times: 78
First time: 0.20873605285547087 seconds
Last time: 1.0448466636005722 seconds
Periods between 0.1 ms and 20 ms (pitch between 50 and 10000 Hz),
with a maximum "period factor" of 1.3:
     Number of periods: 76
     Mean period: 0.00890550928130259 seconds
     Stdev period: 0.0004005988277874362 seconds
     Jitter (local): 1.695%
     Jitter (local, absolute): 150.933E-6 seconds
     Jitter (rap): 0.822%
     Jitter (ppq5): 0.816%
     Jitter (ddp): 2.466%
All periods:
     Number of periods: 77
     Mean period: 0.010858579360325991 seconds
     Stdev period: 0.0171427401057966 seconds
     Jitter (local): 38.053711%
     Jitter (local, absolute): 4132.092366E-6 seconds
     Jitter (rap): 25.495462%
     Jitter (ppq5): 31.168636%
     Jitter (ddp): 76.486386%



PraatError: Command "Jitter (local)" not available for given objects.